In [142]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

### 데이터 로드 (q: 15분 단위, h: 1시간 단위)

In [143]:
generation_q = pd.read_csv("C:/Users/symply_jina/Desktop/seohyun/UIC4801/DATA_generation.csv")
generation_q['Time'] = pd.to_datetime(generation_q['Time'], format='%Y-%m-%d %H:%M')
generation_q['Hour'] = generation_q['Time'].dt.floor('H') 
generation_h = generation_q.groupby('Hour').sum(numeric_only=True)  

demand_q = pd.read_csv("C:/Users/symply_jina/Desktop/seohyun/UIC4801/DATA_demand.csv")
demand_q['Time'] = pd.to_datetime(demand_q['Time'], format='%Y-%m-%d %H:%M')
demand_q['Hour'] = demand_q['Time'].dt.floor('H')   
demand_h = demand_q.groupby('Hour').sum(numeric_only=True)  

price_q = pd.read_csv("C:/Users/symply_jina/Desktop/seohyun/UIC4801/DATA_price.csv")
price_q['Time'] = pd.to_datetime(price_q['Time'], format='%Y-%m-%d %H:%M')

### Set

In [144]:
I = list(range(len(generation_q.columns) - 2))
T = list(generation_q['Time'].dt.hour.unique())
S = list(range(50))
prob = {s: 1 / len(S) for s in S}

### 데이터 전처리

#### Generation

In [145]:
generation_avg = {
    (i, t): generation_h[generation_h.index.hour == t].mean()[i] for i in I for t in T
}

def generate_randomized_generation(I, T, S, generation_avg, randomness_level):
    np.random.seed(7)
    if randomness_level == 'low':
        noise_factors = {(i, t, s): np.random.uniform(0.8, 1.2) for i in I for t in T for s in S}
    elif randomness_level == 'medium':
        noise_factors = {(i, t, s): np.random.uniform(0.5, 1.5) for i in I for t in T for s in S}
    elif randomness_level == 'high':
        noise_factors = {(i, t, s): np.random.uniform(0.2, 1.8) for i in I for t in T for s in S}
    else:
        raise ValueError("Invalid randomness level. Please choose 'low', 'medium', or 'high'.")
    
    generation_r = {
        (i, t, s): generation_avg[(i, t)]*noise_factors[(i, t, s)] for i in I for t in T for s in S
    }
    return generation_r

#### Demand

In [146]:
demand_avg = {
    (i, t): demand_h[demand_h.index.hour == t].mean()[i] for i in I for t in T
}

def generate_randomized_demand(I, T, S, demand_avg, randomness_level):
    np.random.seed(17)
    if randomness_level == 'low':
        noise_factors = {(i, t, s): np.random.uniform(0.8, 1.2) for i in I for t in T for s in S}
    elif randomness_level == 'medium':
        noise_factors = {(i, t, s): np.random.uniform(0.5, 1.5) for i in I for t in T for s in S}
    elif randomness_level == 'high':
        noise_factors = {(i, t, s): np.random.uniform(0.2, 1.8) for i in I for t in T for s in S}
    else:
        raise ValueError("Invalid randomness level. Please choose 'low', 'medium', or 'high'.")
    
    demand_r = {
        (i, t, s): demand_avg[(i, t)]*noise_factors[(i, t, s)] for i in I for t in T for s in S
    }
    return demand_r

#### Randomize

In [147]:
random_key = 'medium'

In [148]:
generation_r = generate_randomized_generation(I, T, S, generation_avg, random_key)
demand_r = generate_randomized_demand(I, T, S, demand_avg, random_key)

#### Residual = generation - demand

In [149]:
residual = {(i, t, s): generation_r[(i, t, s)] - demand_r[(i, t, s)] for i in I for t in T for s in S}

R = {
    (i, t, s): max(0, residual[(i, t, s)])
    for i in I for t in T for s in S
}

#### Price 설정

In [150]:
unique_days = generation_q['Time'].dt.normalize().unique()
days = len(unique_days)
S = list(range(min(len(S), days)))

price_q['Hour'] = price_q['Time'].dt.floor('H')
price_h = price_q.groupby('Hour').mean(numeric_only=True)  

price = price_h.iloc[:len(S)*len(T)]

P_DA = {t: sum(price['Price'].iloc[t + s * len(T)] for s in S) / len(S) * 1.2 for t in T}
P_RT = {(t, s): price['Price'].iloc[t + s * len(T)] for t in T for s in S}
P_PN = {t: sum(price['Price'].iloc[t + s * len(T)] for s in S) / len(S) * 2 for t in T}

### 모델 DER only

In [151]:
x_DA_1 = {(i, t): cp.Variable(pos=True) for i in I for t in T}
y_plus_1 = {(i, t, s): cp.Variable(pos=True) for i in I for t in T for s in S}
y_minus_1 = {(i, t, s): cp.Variable(pos=True) for i in I for t in T for s in S}

In [152]:
obj_case1 = sum(
    P_DA[t] * x_DA_1[i, t] for i in I for t in T
) + sum(
    prob[s] * (P_RT[t, s] * y_plus_1[i, t, s] - P_PN[t] * y_minus_1[i, t, s])
    for i in I for t in T for s in S
)

constraints_case1 = []

for i in I:
    for t in T:
        for s in S:
            constraints_case1.append(R[i, t, s] - x_DA_1[i, t] == y_plus_1[i, t, s] - y_minus_1[i, t, s])

for i in I:
    for t in T:
        for s in S:
            constraints_case1.append(R[i,t,s] >= y_plus_1[i, t, s])


M = max(R[i,t,s] for i in I for t in T for s in S)
z_case1 = {(i, t, s): cp.Variable(boolean=True) for i in I for t in T for s in S}
for i in I:
    for t in T:
        for s in S:
            constraints_case1.append(y_plus_1[i, t, s] <= M * z_case1[i, t, s])
            constraints_case1.append(y_minus_1[i, t, s] <= M * (1 - z_case1[i, t, s]))

prob_case1 = cp.Problem(cp.Maximize(obj_case1), constraints_case1)
prob_case1.solve(solver='GUROBI', IntFeasTol=1e-9, TimeLimit=60*30)

c:\Users\symply_jina\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\problems\problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


10992.730232353653

In [153]:
# 결과 저장
data = {
    i: [x_DA_1[i, t].value for t in T] for i in I
}
df = pd.DataFrame(data, index=T)
df.to_excel(f'./x_DA_1_results_rand{random_key}.xlsx')

output_file = f"./y_plus_1_results_rand{random_key}.xlsx"

with pd.ExcelWriter(output_file) as writer:
    for i in I:
        data = {
            s: [y_plus_1[i, t, s].value for t in T] for s in S
        }
        df = pd.DataFrame(data, index=T)
        
        sheet_name = f"i_{i}"
        df.to_excel(writer, sheet_name=sheet_name)
        
output_file = f"./y_minus_1_results_rand{random_key}.xlsx"

with pd.ExcelWriter(output_file) as writer:
    for i in I:
        data = {
            s: [y_minus_1[i, t, s].value for t in T] for s in S
        }
        df = pd.DataFrame(data, index=T)
        
        sheet_name = f"i_{i}"
        df.to_excel(writer, sheet_name=sheet_name)

### 결과 분석

#### 수익 분석

In [154]:
# Day-ahead 수익 비교
total_da_profit_obj = 0
for i in I:
    for t in T:
        total_da_profit_obj += P_DA[t] * x_DA_1[i,t].value

# Real-time 수익 비교
total_rt_profit_obj = 0
for i in I:
    for t in T:
        for s in S:
            rt_profit_obj = P_RT[t, s] * y_plus_1[i, t, s].value
            total_rt_profit_obj += prob[s] * rt_profit_obj

# 패널티 비용 비교
total_penalty_cost_obj = 0
for i in I:
    for t in T:
        for s in S:
            penalty_cost_obj = P_PN[t] * y_minus_1[i, t, s].value
            total_penalty_cost_obj += prob[s] * penalty_cost_obj

# 총 시스템 이익 (목적 함수 기반)
total_system_profit_obj = total_da_profit_obj + total_rt_profit_obj - total_penalty_cost_obj

# 결과 출력
print(f"총 Day-ahead 수익 (_obj): {total_da_profit_obj:.2f}")
print(f"총 Real-time 수익 (_obj): {total_rt_profit_obj:.2f}")
print(f"총 Penalty 비용 (_obj): {total_penalty_cost_obj:.2f}")
print(f"목적 함수 기반 총 이익 (_obj): {total_system_profit_obj:.2f}")

총 Day-ahead 수익 (_obj): 2945.38
총 Real-time 수익 (_obj): 8618.07
총 Penalty 비용 (_obj): 570.72
목적 함수 기반 총 이익 (_obj): 10992.73


#### 하루 개인별 커밋량 합 분석

In [155]:
sum_x = 0
for t in T:
    time_sum = sum(x_DA_1[i,t].value for i in I)
    print(f"시간 {t}: {time_sum}")
    sum_x += time_sum
print(f"총 하루 commmit: {sum_x}")


시간 0: 0.0
시간 1: 0.0
시간 2: 0.0
시간 3: 0.0
시간 4: 0.0
시간 5: 0.0
시간 6: 0.0
시간 7: 0.0
시간 8: 0.0
시간 9: 0.2705125471686891
시간 10: 6.633969304838254
시간 11: 18.57972207988184
시간 12: 21.14673408322065
시간 13: 25.77011898934513
시간 14: 13.002038707067452
시간 15: 5.538486093819148
시간 16: 0.7208828767958564
시간 17: 0.0
시간 18: 0.0
시간 19: 0.0
시간 20: 0.0
시간 21: 0.0
시간 22: 0.0
시간 23: 0.0
총 하루 commmit: 91.66246468213703


#### 개인 수익 분석

In [156]:
# 모든 der의 profit 합계 계산
total_der_profit = 0
der_profit = {}
for i in I:
    # 각 der i의 profit 합계 계산
    der_profit[i] = sum(P_DA[t] * x_DA_1[i,t].value + sum(prob[s] * (P_RT[t,s] * y_plus_1[i,t,s].value - P_PN[t] * y_minus_1[i,t,s].value) for s in S) for t in T)
    total_der_profit += der_profit[i]

print("\nder_profit:")
for i, value in der_profit.items():
    print(f"{i}: {value:.2f}")

print(f"\n모든 der의 profit 합계: {total_der_profit:.2f}")



der_profit:
0: 912.57
1: 595.73
2: 941.72
3: 824.58
4: 547.22
5: 511.60
6: 1159.65
7: 820.22
8: 423.18
9: 606.98
10: 371.08
11: 87.75
12: 664.98
13: 533.54
14: 581.47
15: 467.51
16: 91.94
17: 498.09
18: 352.92

모든 der의 profit 합계: 10992.73


### 결과 저장

In [157]:
R_sum = {(i, t): sum(R[i, t, s] for s in S) / len(S) for i in I for t in T}
R_df = pd.DataFrame({i: {t: R_sum[i, t] for t in T} for i in I})
R_df.to_csv('result_R.csv', index=False)

pd.DataFrame(list(P_DA.items()), columns=['time', 'price']).to_csv('result_P_DA.csv', index=False)

total_x_da_1 = {t: sum(x_DA_1[i,t].value for i in I) for t in T}
total_x_da_1_df = pd.DataFrame({'total_x_da_1': total_x_da_1})
total_x_da_1_df.to_csv('result_total_X_DA.csv', index=False)

der_profit_value = pd.DataFrame({'der_profit_value': der_profit})
der_profit_value.to_csv('result_der_profit.csv', index=False)

only_value = pd.DataFrame({'only_value': [prob_case1.value]})
only_value.to_csv('result_only_value.csv', index=False)